In [1]:
import os
import pandas as pd
import numpy as np
from importlib import reload

import data_wrangle
import spectral
import analysis
import __utils__

from prettytable import PrettyTable as pt

#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

from math import ceil
from collections import Counter

# import plotly.io as pio
# pio.renderers.default = "browser"
# pio.renderers.default = "png"

# Initalize Object and set constants

In [2]:
path='/mnt/c/Users/16162/Desktop/dopamine.db'
table='arsenic'

## Look at the Table

In [174]:
reload(data_wrangle)
db=data_wrangle.DataTable(path,table)
# db.summary() # fix column info

db.__del__()

# Grab data
---

+ Can remove spectra by id:  
`db.deab_list.append(#)`   
+ Can set conditions:  
    + `db.condition='WHERE col IS NOT NULL'`  
    + `db.condition='WHERE col IS value'`  
    + `db.condition='WHERE col1 IS NOT NULL AND col2 = value'`
    + `db.condition='WHERE strftime(\'%Y\',date)=\'2023\' AND general_loc in (\'substantia nigra\',\'cortex\',\'midbrain\')'`  
    + `db.condition="WHERE date LIKE '2023-%'"# AND location IN ('cortex')"`
+ As currently write, `raman_shifts()` should be executed twice to automatically remove problematic Raman Shifts

In [4]:
reload(__utils__)
reload(data_wrangle)
db=data_wrangle.DataTable(path, table)

#1) Get X and Y data
x=db.raman_shifts()
ys=db.intens()

#2) Generate labeling dictionaries
namedic=db.label_dict(cols=['As3', 'As5'])

__utils__.dict_summary(namedic)

There are 2 Ramanshift lists found in the dataset.
+----------------------------------------------+
|                   Outliers                   |
+----------------------------------------------+
| There are no outliers in the remaining data! |
+----------------------------------------------+
+----------------------------------------------+


('There are 225 entires and 9 unique values in the dictionary.',
 +-----------------------+-------------------+
 |       Dict Value      | Frequency in Dict |
 +-----------------------+-------------------+
 | 50 uM As3, 150 uM As5 |         25        |
 |      0 As3, 0 As5     |         25        |
 |  19 uM As3, 1 mM As5  |         25        |
 |    50 uM As3, 0 As5   |         25        |
 | 19 uM As3, 150 uM As5 |         25        |
 |    19 uM As3, 0 As5   |         25        |
 |    0 As3, 1 mM As5    |         25        |
 |  50 uM As3, 1 mM As5  |         25        |
 |   0 As3, 150 uM As5   |         25        |
 +-----------------------+-------------------+)

# View spectra

In [ ]:
#3) Make color dictionary
for item in set(namedic.values()):
    print(f'\'{item}\':\'#\',',)

#4) Make set dictionary

In [85]:
color_dic={
'19 uM As3, 1 mM As5':'#8200ff',
'0 As3, 150 uM As5':'#6600ff',
'0 As3, 1 mM As5':'#250094',
'19 uM As3, 0 As5':'#ff0061',
'0 As3, 0 As5':'grey',
'19 uM As3, 150 uM As5':'#da00c6',
'50 uM As3, 0 As5':'#b40000',
'50 uM As3, 1 mM As5':'#a800e7',
'50 uM As3, 150 uM As5':'#ff00ae',
}

color_dic={
'0 As3, 1 mM As5':'#250094',
'0 As3, 150 uM As5':'#00eaff',
'50 uM As3, 150 uM As5':'#824c96',
'19 uM As3, 150 uM As5':'#433466',
'50 uM As3, 1 mM As5':'#ff7a00',
'19 uM As3, 1 mM As5':'#ed733f',
'19 uM As3, 0 As5':'#ff7a00',
'50 uM As3, 0 As5':'#ff0000',
'0 As3, 0 As5':'grey',}

set_dic={
  'Mix':['50 uM As3, 1 mM As5', '19 uM As3, 1 mM As5', '19 uM As3, 150 uM As5', '50 uM As3, 150 uM As5'],
  'As3':['50 uM As3, 0 As5','19 uM As3, 0 As5'],
  'As5':['0 As3, 1 mM As5','0 As3, 150 uM As5'],
  'Blank':['0 As3, 0 As5']
 }

In [121]:
#testing
reload(spectral)


spec=spectral.Spectral(ys,x,namedic,set_dic,color_dict=color_dic,
                       subplot_height=100)
spec.trunc_before_x=314


spec.set_plot(move_y_title=0.08)



In [118]:
for i in fig['data']:
    print(color_dic[i['name']])

#ed733f
#433466
#824c96
#ff7a00
#ff7a00
#ff0000
#00eaff
#250094
grey


# PCA

In [123]:
reload(analysis)
var=analysis.multivar(ys,x,namedic, color_dict=color_dic)
# pc,a,b=var.pca_run()
# var.pca_fig('PC 1', 'PC 2')
# var.pca_fig('PC 1', 'PC 3')
# var.pca_fig('PC 2', 'PC 3')
var.pca_3d()


dictionary for color labels exists


In [124]:
ls=var.loadings('1','2', '3')
for l in ls:
    l.show()


# Extras!

## Update Database Find and Replace

In [31]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
db.condition='WHERE sheep IS 2'
db.data_info('nanotag') #Add 'condition' to find_replace
# db.find_replace('BoooTh', 'Booo', 'nanotag')

Number of samples selected: 75


[+-------------------+
 |      nanotag      |
 +-------+-----------+
 | Value | Frequency |
 +-------+-----------+
 |  Booo |     69    |
 |  NaN  |     1     |
 |  nba  |     1     |
 |   no  |     4     |
 +-------+-----------+]

## Making Figure - old code

In [ ]:
reload(__utils__)
#This should go into spectral
def truncate(x_data, y_data, before_target=None, after_target=None):
    '''targets need to be intergers for the Raman Shift values.
    x_data = Raman Shifts as a series or list
    y_data = df where each row is a sample
    If no only 1 target is given, this should still work'''#***
    #Find indexes for truncating
    if before_target==None:
        before_idx=0
    else:
        before_idx=__utils__.find_index(x_data, before_target)
    if after_target==None:
        after_idx=len(x_data)
    else:
        after_idx=__utils__.find_index(x_data, after_target)
    print(after_idx,'FIX return error if NONE')
    res_x=x_data[before_idx:after_idx]
    res_ys=y_data.iloc[:,before_idx:after_idx]
    t1=go.Scatter(x=x_data, y=y_data.iloc[1], name='before')
    t2=go.Scatter(x=res_x, y=res_ys.iloc[1], name='after')
    traces=[t1,t2]
    fig=go.Figure(traces)
    fig.update_layout(title='Visualize Data Truncation')
    fig.show()

    return res_x, res_ys

# x,y=truncate(x,ys,500,1080)

## Find and remove NaN values

In [ ]:
#Find NaN values
nan_cols=ys.isna().any() #Find columns with NaN values
nan_rows=ys.loc[:,nan_cols].isna().any(axis=1) #Find the rws with NaN values (in columns with NaN values)
nan_rows.index #Get indexes of rows (= sample ID)

#Drop rows with more than 5 NaN values
ys=ys.dropna(thresh=ys.shape[1]-5)

#Map namedic to sample ID
tag=ys.index.map(lambda x: namedic[x][0])

## Something to do with dataframe manipulation

In [ ]:
df=pd.DataFrame([[1,2,3,4,5],[2,3,4,5,6],[1,2,3,4,5],[5,6,7,8,9],[1,2,3,4,5],[5,6,7,8,9]])
print(df)
uniques=set(map(tuple,df.values))
uniques=list(uniques)
for u in uniques:
    mask=np.all(df==u, axis=1)
    print(mask)
print(df[mask])
print(df)


   0  1  2  3  4
0  1  2  3  4  5
1  2  3  4  5  6
2  1  2  3  4  5
3  5  6  7  8  9
4  1  2  3  4  5
5  5  6  7  8  9
0    False
1    False
2    False
3     True
4    False
5     True
dtype: bool
0     True
1    False
2     True
3    False
4     True
5    False
dtype: bool
0    False
1     True
2    False
3    False
4    False
5    False
dtype: bool
   0  1  2  3  4
1  2  3  4  5  6
   0  1  2  3  4
0  1  2  3  4  5
1  2  3  4  5  6
2  1  2  3  4  5
3  5  6  7  8  9
4  1  2  3  4  5
5  5  6  7  8  9
